In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 
import time
from ipywidgets import FloatProgress
from IPython.display import display
from bs4 import BeautifulSoup
import requests
import re
from os import path
from PIL import Image
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from models import *
from etl import *
import plotly.plotly as py
import plotly.graph_objs as go

Database session created


In [2]:
def add_scraping_source(df, identifier_dict):
    scraped_source = []
    for index, row in df.iterrows():
        identifier = row.medium + row.formality
        scraped_from = identifier_dict[identifier]
        scraped_source.append(scraped_from)
    df['scraped_from'] = scraped_source
    return df
def fix_NYT(df):
    df.loc[(df.provider=='The New York Times') & (df.medium == 'text'), 'scraped_from'] = 'NYT API'
    return df

In [3]:
identifier_dict = {'audioFormal':'Itunes Audiobook','textFormal':'Itunes Ebook','videoFormal':'Itunes Movie',
                  'textInformal': 'Twitter', 'videoInformal': 'Youtube',
                   'audioIntermediate': 'Itunes Podcast', 'textIntermediate': 'NewsApi', 'videoIntermediate': 'Youtube'}

In [4]:
pd.set_option('display.max_colwidth', -1)

In [5]:
df = pd.DataFrame([{'id':content.id,'content_url':content.content_url, 'image_url':content.image_url,'title':content.title
                             , 'published': content.published, 'length': content.length, 'medium':content.medium.name,
                             'provider': content.provider.name, 'api_id':content.provider.api_id,
                    'formality':content.provider.formality.type, 'difficulty':content.difficulty.type
                             } for content in session.query(Content).all()])
x = [[category.name for category in content.categories] for content in session.query(Content).all()]
df['categories'] = x
df = add_scraping_source(df, identifier_dict)
df = fix_NYT(df)

df.to_csv('all_content.csv')

In [6]:
df

,api_id,content_url,difficulty,formality,id,image_url,length,medium,provider,published,title,categories,scraped_from
0,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP010.mp3,Hard,Intermediate,1,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,2705.000,audio,The Microbiome Podcast,2015-10-19,Episode 10: The microbiome and gut-brain communications with Dr. John Cryan,"[gut, microbiome, null]",Itunes Podcast
1,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP009.mp3,Hard,Intermediate,2,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,3307.000,audio,The Microbiome Podcast,2015-06-15,Episode 9: The skin microbiome and AOBiome with Dr. Larry Weiss,"[microbiome, null]",Itunes Podcast
2,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP008.mp3,Medium,Intermediate,3,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,2828.000,audio,The Microbiome Podcast,2015-06-01,Episode 8: The vaginal microbiome with Dr. Jacques Ravel,"[microbiome, null]",Itunes Podcast
3,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP007.mp3,Medium,Intermediate,4,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,2522.000,audio,The Microbiome Podcast,2015-05-18,Episode 7: Gut bacteria and circadian clocks with Drs. Eugene Chang and Vanessa Leone,"[gut, microbiome, null]",Itunes Podcast
4,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP006.mp3,Hard,Intermediate,5,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,3180.000,audio,The Microbiome Podcast,2015-05-04,"Episode 6: The Microbiome, autism, and serotonin production with Dr. Elaine Hsiao","[gut, microbiome, null]",Itunes Podcast
5,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP005.mp3,Easy,Intermediate,6,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,3123.000,audio,The Microbiome Podcast,2015-04-21,Episode 5: Diet and its impact on our microbiota and health with Drs. Erica and Justin Sonnenburg,"[gut, microbiome, null, fiber, mood]",Itunes Podcast
6,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP004.mp3,Easy,Intermediate,7,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,2764.000,audio,The Microbiome Podcast,2015-04-06,Episode 4: What's hot in the microbiome field with Kristina Campbell,"[gut, microbiome, null]",Itunes Podcast
7,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP003.mp3,Medium,Intermediate,8,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,3007.000,audio,The Microbiome Podcast,2015-03-23,Episode 3: Emulsifiers in our food with Dr. Andrew Gewirtz,"[microbiome, null]",Itunes Podcast
8,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP002.mp3,Easy,Intermediate,9,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,2649.000,audio,The Microbiome Podcast,2015-03-08,Episode 2: All things microbiome with Dr. Jonathan Eisen,"[microbiome, null]",Itunes Podcast
9,Itunes Podcast,http://traffic.libsyn.com/microbiome/TMP001S.mp3,Easy,Intermediate,10,http://static.libsyn.com/p/assets/8/0/d/a/80da08206f966c8f/1400x1400-ami-microbiome-podcast.png,786.000,audio,The Microbiome Podcast,2015-02-23,Episode 1 Supplement: Microbiome of our transit systems with Dr. Chris Mason,"[microbiome, null]",Itunes Podcast


In [7]:
# x = [[category.name for category in content.categories] for content in session.query(Content).all()]
# y = [l for l in x if len(l) > 2]

# Total Content Per Medium & Formality

In [7]:
total = df[['content_url','formality','medium','api_id']].groupby(['formality','medium','api_id']).nunique()
total[['content_url']]

content_url
formality    medium api_id                             
Formal       audio  Itunes Audiobook        1017       
                    Itunes Ebook            284        
             text   Itunes Ebook            2961       
             video  Itunes Ebook            74         
                    Itunes Movie            114        
Informal     text   Facebook                6          
                    Twitter                 1674       
             video  Facebook                8          
                    Instagram               14         
                    Twitter                 3          
Intermediate audio  Itunes Podcast          7109       
             text   Blog                    49         
                    NewsAPI                 561        
                    The New York Times API  828        
             video  Itunes Podcast          53         
                    NewsAPI                 1          
                    The New York Times API  11         
                    Youtube                 2635

In [8]:
total = df[['content_url','formality','medium','difficulty','api_id']].groupby(['formality','medium','difficulty','api_id']).nunique()
total[['content_url']]

content_url
formality    medium difficulty api_id                             
Formal       audio  Easy       Itunes Audiobook        332        
                               Itunes Ebook            84         
                    Hard       Itunes Audiobook        341        
                               Itunes Ebook            103        
                    Medium     Itunes Audiobook        344        
                               Itunes Ebook            97         
             text   Easy       Itunes Ebook            964        
                    Hard       Itunes Ebook            1008       
                    Medium     Itunes Ebook            989        
             video  Easy       Itunes Ebook            22         
                               Itunes Movie            41         
                    Hard       Itunes Ebook            22         
                               Itunes Movie            27         
                    Medium     Itunes Ebook            30         
                               Itunes Movie            46         
Informal     text   Easy       Facebook                2          
                               Twitter                 574        
                    Hard       Facebook                2          
                               Twitter                 585        
                    Medium     Facebook                2          
                               Twitter                 515        
             video  Easy       Facebook                2          
                               Instagram               1          
                               Twitter                 2          
                    Hard       Facebook                4          
                               Instagram               5          
                               Twitter                 1          
                    Medium     Facebook                2          
                               Instagram               8          
Intermediate audio  Easy       Itunes Podcast          2355       
                    Hard       Itunes Podcast          2403       
                    Medium     Itunes Podcast          2351       
             text   Easy       Blog                    19         
                               NewsAPI                 179        
                               The New York Times API  270        
                    Hard       Blog                    15         
                               NewsAPI                 192        
                               The New York Times API  267        
                    Medium     Blog                    15         
                               NewsAPI                 190        
                               The New York Times API  296        
             video  Easy       Itunes Podcast          16         
                               The New York Times API  4          
                               Youtube                 852        
                    Hard       Itunes Podcast          20         
                               The New York Times API  4          
                               Youtube                 865        
                    Medium     Itunes Podcast          17         
                               NewsAPI                 1          
                               The New York Times API  3          
                               Youtube                 918

# Total Unique Categories and Providers per Medium & Formality

In [9]:
plot_stacked_OVERALL()

# Ketogenic

In [11]:
Limit_widget1 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget1 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget1 = widgets.Dropdown(options = nutrition_cats,value='keto', description ='Topic', disabled=False)
expertise_widget1 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget1 = widgets.IntSlider(min=0, max=100, step=10, value=20)
difficulty_widget1 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')


interact(query_content,Limit = Limit_widget1, Medium = Medium_widget1, search_param=topic_widget1, Formality = expertise_widget1, Max_Length = max_length_widget1,Difficulty = difficulty_widget1);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [11]:
alt_widget_1 = widgets.Dropdown(options = nutrition_cats,value='keto', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_1);

interactive(children=(Dropdown(description='Topic', options=('keto', 'mct', 'natural', 'corn-fed', 'cruciferou…

# Paleolithic

In [12]:
Limit_widget2 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget2 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget2 = widgets.Dropdown(options = nutrition_cats,value='paleo', description ='Topic', disabled=False)
expertise_widget2 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget2 = widgets.IntSlider(min=0, max=100, step=10, value=20)
difficulty_widget2 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')


interact(query_content,Limit = Limit_widget2, Medium = Medium_widget2, search_param=topic_widget2, Formality = expertise_widget2, Max_Length = max_length_widget2,Difficulty = difficulty_widget2);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [13]:
alt_widget_2 = widgets.Dropdown(options = nutrition_cats,value='paleo', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_2);

interactive(children=(Dropdown(description='Topic', index=14, options=('keto', 'mct', 'natural', 'corn-fed', '…

# Vegan

In [13]:
Limit_widget3 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget3 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget3 = widgets.Dropdown(options = nutrition_cats,value='vegan', description ='Topic', disabled=False)
expertise_widget3 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget3 = widgets.IntSlider(min=0, max=100, step=10, value=20)
difficulty_widget3 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')


interact(query_content,Limit = Limit_widget3, Medium = Medium_widget3, search_param=topic_widget3, Formality = expertise_widget3, Max_Length = max_length_widget3, Difficulty = difficulty_widget3);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [14]:
alt_widget_3 = widgets.Dropdown(options = nutrition_cats,value='vegan', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_3);

interactive(children=(Dropdown(description='Topic', index=13, options=('keto', 'mct', 'natural', 'corn-fed', '…

# Vegetarian

In [15]:
Limit_widget4 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget4 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget4 = widgets.Dropdown(options = nutrition_cats,value='vegetarian', description ='Topic', disabled=False)
expertise_widget4 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget4 = widgets.IntSlider(min=0, max=200, step=10, value=20)
difficulty_widget4 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')

interact(query_content,Limit = Limit_widget4, Medium = Medium_widget4, search_param=topic_widget4, Formality = expertise_widget4, Max_Length = max_length_widget4, Difficulty = difficulty_widget4);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [16]:
alt_widget_4 = widgets.Dropdown(options = nutrition_cats,value='vegetarian', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_4);

interactive(children=(Dropdown(description='Topic', index=14, options=('keto', 'mct', 'natural', 'corn-fed', '…

# General

In [17]:
Limit_widget5 = widgets.IntSlider(min=1, max=5, step=1, value=1)
Medium_widget5 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')
topic_widget5 = widgets.Dropdown(options = ['general'],value='general', description ='Topic', disabled=False)
expertise_widget5 = widgets.ToggleButtons(options = ['Informal', 'Intermediate','Formal'], description ='Formality', button_style='')
max_length_widget5 = widgets.IntSlider(min=0, max=300, step=10, value=20)
difficulty_widget5 = widgets.ToggleButtons(options = ['Easy', 'Medium','Hard'], description ='Difficulty', button_style='')

interact(query_content,Limit = Limit_widget5, Medium = Medium_widget5, search_param=topic_widget5, Formality = expertise_widget5, Max_Length = max_length_widget5,Difficulty - difficulty_widget5);

interactive(children=(IntSlider(value=1, description='Limit', max=5, min=1), ToggleButtons(description='Medium…

In [18]:
alt_widget_5 = widgets.Dropdown(options = ['general'],value='general', description ='Topic', disabled=False)

interact(plot_stacked, param = alt_widget_5);

interactive(children=(Dropdown(description='Topic', options=('general',), value='general'), Output()), _dom_cl…

# Search

In [19]:
# search_widget = widgets.Text(placeholder = 'Search News', description='Search:', disabled=False,continuous_update=False)
# Limit_widget5 = widgets.IntSlider(min=1, max=5, step=1, value=1)
# Medium_widget5 = widgets.ToggleButtons(options = ['text', 'audio','video'], description ='Medium', button_style='')




# interact(query_content,search_param=search_widget,Limit = Limit_widget5, Medium = Medium_widget5);